In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import os

In [2]:
# Used https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/
def positionalEncoding(seq_len, d, n=10000):
    p = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            p[k, 2*i] = np.sin(k/denominator)
            p[k, 2*i+1] = np.cos(k/denominator)
    return p

In [3]:
# Generate EG3D Images
from eg3d.camera_utils import LookAtPoseSampler, FOV_to_intrinsics
device = 'cuda'
cam2world_pose = LookAtPoseSampler.sample(3.14/2, 3.14/2, torch.tensor([0, 0, 0.2], device=device), radius=2.7, device=device)
intrinsics = FOV_to_intrinsics(18, device=device)

with open('eg3d/eg3d_model/ffhqrebalanced512-128.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module
c = torch.cat([cam2world_pose.reshape(-1, 16), intrinsics.reshape(-1, 9)], 1) # camera parameters
# z = torch.randn([1, G.z_dim]).cuda()    # latent codes
z = torch.tensor(znprecon).view(-1, 512).cuda()

# img = G(z, c)['image']                           # NCHW, float32, dynamic range [-1, +1], no truncation

cam_pivot = torch.tensor(G.rendering_kwargs.get('avg_camera_pivot', [0, 0, 0]), device=device)
cam_radius = G.rendering_kwargs.get('avg_camera_radius', 2.7)
cam2world_pose = LookAtPoseSampler.sample(np.pi/2, np.pi/2, cam_pivot, radius=cam_radius, device=device)
conditioning_cam2world_pose = LookAtPoseSampler.sample(np.pi/2, np.pi/2, cam_pivot, radius=cam_radius, device=device)
camera_params = torch.cat([cam2world_pose.reshape(-1, 16), intrinsics.reshape(-1, 9)], 1)
conditioning_params = torch.cat([conditioning_cam2world_pose.reshape(-1, 16), intrinsics.reshape(-1, 9)], 1)
w = G.mapping(z, conditioning_params, truncation_psi=0.5, truncation_cutoff=8)
img = G.synthesis(w, camera_params)['image']


ModuleNotFoundError: No module named 'training'

In [ ]:
cpu_img = img.cpu().numpy()[0].transpose(1, 2, 0)
plt.imshow(cpu_img)
plt.show()

print(np.amax(cpu_img[0]))

In [ ]:
np.amin(z.cpu().numpy())

In [ ]:
size = G.z_dim
num = 10

A = positionalEncoding(size, size)

znp = np.tile(np.reshape(z[0].cpu().numpy(), (size, 1)), size)
summed_matrix = A + znp

znprecon = np.nanmean(product-A, axis=1)
print(product[0][1])
print(znprecon[0])

matshow_prod = plt.matshow(summed_matrix)
plt.gcf().colorbar(matshow_prod)
plt.show()
